In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro'}
band = [7]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.037772                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.030199
0.000750      2.0 -0.042215                  2.0 -0.033779
0.001052      3.0 -0.050754                  3.0 -0.040741
0.001476      4.0 -0.058853                  4.0 -0.047393
0.002070      5.0 -0.066510                  5.0 -0.053744
0.002904      6.0 -0.073718                  6.0 -0.059784
0.004074      7.0 -0.080466                  7.0 -0.065513
0.005714      8.0 -0.086743                  8.0 -0.070902
0.008015      9.0 -0.092543                  9.0 -0.075957
0.011243     10.0 -0.097972                 10.0 -0.080682
0.015771     11.0 -0.103111                 11.0 -0.085073
0.022122     12.0 -0.108045                 12.0 -0.089135
0.031031     13.0 -0.112937                 13.0 -0.092893
0.043528     14.0 -0.116866                 14.0 -0.095594
0.061057     15.0 -0.113600                 15.0 -0.093670
0.085645     16.0 -0.081902                 16.0 -0.071034
0.120136     17.0 -0.000288                 17.0 -0.007916
0.168516     18.0  0.138644                 18.0  0.106138
0.236378     19.0  0.344954                 19.0  0.283884
0.331549     20.0  0.629648                 20.0  0.539565
0.465100     21.0  1.006924                 21.0  0.888142
0.652400     22.0  1.447895                 22.0  1.301872
0.915100     23.0  1.722888                 23.0  1.599929
1.283650     24.0  1.709178                 24.0  1.652862
1.800600     25.0  1.539786                 25.0  1.512629
2.525700     26.0  1.224873                 26.0  1.185750
3.542800     27.0  0.873641                 27.0  0.834189
4.969550     28.0  0.589380                 28.0  0.567499
6.970850     29.0  0.383766                 29.0  0.385706
9.778100     30.0  0.188196                 30.0  0.207726
13.715850    31.0 -0.046761                 31.0 -0.030245
19.239350    32.0 -0.267163                 32.0 -0.266001
26.987250    33.0 -0.445554                 33.0 -0.466240
37.855300    34.0 -0.540220                 34.0 -0.593104
53.100050    35.0 -0.508813                 35.0 -0.577510
73.887500    36.0 -0.342007                 36.0 -0.377347
97.662500    37.0 -0.171369                 37.0 -0.179324
121.437500   38.0 -0.108363                 38.0 -0.110397
145.212500   39.0 -0.089490                 39.0 -0.090322
168.987500   40.0 -0.075971                 40.0 -0.075994
192.762500   41.0 -0.065306                 41.0 -0.064827
216.537500   42.0 -0.055300                 42.0 -0.054604
240.312500   43.0 -0.046670                 43.0 -0.045938
264.087500   44.0 -0.039383                 44.0 -0.038701
287.862500   45.0 -0.033342                 45.0 -0.032765
311.637500   46.0 -0.028376                 46.0 -0.027936
335.412500   47.0 -0.024003                 47.0 -0.023706
359.187500   48.0 -0.019891                 48.0 -0.019780
382.962500   49.0 -0.016057                 49.0 -0.016154
406.737500   50.0 -0.012424                 50.0 -0.012729
430.512500   51.0 -0.008758                 51.0 -0.009265
454.287500   52.0 -0.004968                 52.0 -0.005679
478.062500   53.0 -0.001012                 53.0 -0.001911
501.837500   54.0  0.003143                 54.0  0.002089
525.612500   55.0  0.007541                 55.0  0.006307
549.387500   56.0  0.012213                 56.0  0.010809
573.162500   57.0  0.017198                 57.0  0.015627
596.937500   58.0  0.022542                 58.0  0.020795
620.712500   59.0  0.028256                 59.0  0.026332
644.487500   60.0  0.034435                 60.0  0.032269
668.262500   61.0  0.041033                 61.0  0.038611
692.037500   62.0  0.047715                 62.0  0.045183
715.812500   63.0  0.054103                 63.0  0.051739
739.587500   64.0  0.061928                 64.0  0.060081
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -26.145960  0.000000 -26.145960        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -25.906921  0.000002   
6.244000e-04 2     -26.145964  0.000002 -26.145963 -25.906925  0.000003   
8.759000e-04 3     -26.145966  0.000002 -26.145964 -25.906926  0.000004   
1.228600e-03 4     -26.145970  0.000004 -26.145966 -25.906929  0.000004   
1.723400e-03 5     -26.145976  0.000006 -26.145970 -25.906933  0.000006   
2.417400e-03 6     -26.145985  0.000010 -26.145975 -25.906940  0.000009   
3.390900e-03 7     -26.145999  0.000016 -26.145984 -25.906951  0.000013   
4.756500e-03 8     -26.146022  0.000026 -26.145997 -25.906969  0.000020   
6.672000e-03 9     -26.146057  0.000041 -26.146016 -25.906996  0.000031   
9.358900e-03 10    -26.146111  0.000065 -26.146046 -25.907038  0.000048   
1.312780e-02 11    -26.146193  0.000103 -26.146089 -25.907101  0.000075   
1.841450e-02 12    -26.146316  0.000162 -26.146154 -25.907196  0.000117   
2.583020e-02 13    -26.146501  0.000252 -26.146249 -25.907339  0.000182   
3.623230e-02 14    -26.146778  0.000390 -26.146388 -25.907552  0.000281   
5.082340e-02 15    -26.147190  0.000599 -26.146590 -25.907869  0.000432   
7.129060e-02 16    -26.147831  0.000966 -26.146866 -25.908364  0.000700   
1.000000e-01 17    -26.148867  0.001723 -26.147144 -25.909168  0.001262   
1.402710e-01 18    -26.150451  0.003306 -26.147145 -25.910403  0.002460   
1.967600e-01 19    -26.152678  0.006461 -26.146218 -25.912139  0.004907   
2.759970e-01 20    -26.155482  0.012503 -26.142980 -25.914300  0.009732   
3.871000e-01 21    -26.158355  0.023661 -26.134693 -25.916414  0.018948   
5.431000e-01 22    -26.159690  0.043605 -26.116086 -25.917086  0.036034   
7.617000e-01 23    -26.155891  0.077298 -26.078593 -25.913364  0.066027   
1.068500e+00 24    -26.144644  0.128665 -26.015979 -25.903288  0.114103   
1.498800e+00 25    -26.126319  0.197461 -25.928858 -25.887054  0.182127   
2.102400e+00 26    -26.100312  0.281550 -25.818762 -25.864293  0.267532   
2.949000e+00 27    -26.070621  0.374696 -25.695925 -25.839013  0.361177   
4.136600e+00 28    -26.044610  0.471589 -25.573022 -25.818572  0.458102   
5.802500e+00 29    -26.031075  0.574360 -25.456715 -25.811909  0.563440   
8.139200e+00 30    -26.044589  0.694100 -25.350489 -25.834075  0.692380   
1.141700e+01 31    -26.117541  0.840125 -25.277416 -25.917918  0.856886   
1.601470e+01 32    -26.297452  0.994568 -25.302884 -26.110759  1.033254   
2.246400e+01 33    -26.636621  1.129633 -25.506988 -26.465095  1.184353   
3.151050e+01 34    -27.216240  1.231787 -25.984453 -27.074348  1.293921   
4.420010e+01 35    -28.082741  1.286244 -26.796498 -28.020317  1.348259   
6.200000e+01 36    -29.147105  1.277763 -27.869342 -29.228210  1.338332   
8.577500e+01 37    -30.073793  1.241250 -28.832543 -30.255360  1.302642   
1.095500e+02 38    -30.538733  1.223559 -29.315173 -30.743140  1.285337   
1.333250e+02 39    -30.841641  1.221284 -29.620358 -31.050502  1.281753   
1.571000e+02 40    -31.104298  1.231908 -29.872390 -31.312475  1.289324   
1.808750e+02 41    -31.338113  1.251764 -30.086349 -31.542359  1.305163   
2.046500e+02 42    -31.549188  1.278915 -30.270273 -31.747442  1.327653   
2.284250e+02 43    -31.737566  1.311551 -30.426015 -31.928858  1.355271   
2.522000e+02 44    -31.906055  1.348601 -30.557454 -32.090052  1.387077   
2.759750e+02 45    -32.057882  1.389514 -30.668368 -32.234511  1.422529   
2.997500e+02 46    -32.196524  1.434254 -30.762269 -32.365826  1.461559   
3.235250e+02 47    -32.325473  1.483288 -30.842185 -32.487517  1.504565   
3.473000e+02 48    -32.446835  1.537051 -30.909784 -32.601560  1.551838   
3.710750e+02 49    -32.561632  1.595829 -30.965804 -32.709090  1.603654   
3.948500e+02 50    -32.671174  1.660148 -31.011025 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')